quick test to validate if all boards that were found with the old scan , are found with the new scan and are not missing from the database 

-  load the old data 
- remove all items that are < v1.18.0
- for each row 
  - check if the long name can be found in the database
    - check the version 
  - check if the short name can be found in the database
    - check the version 

>

In [4]:
import sqlite3
from mpflash.config import config
from mpflash.mpboard_id import read_known_boardinfo
from packaging.version import Version


candidate_boards = read_known_boardinfo()

candidate_boards = [board for board in candidate_boards if Version(board.version) >= Version("1.18.0")]

In [9]:
from pathlib import Path

missing = []

with sqlite3.connect(config.db_path) as conn:
    conn.row_factory = sqlite3.Row
    for board in candidate_boards:
        # print(f'Checking {board.description}')
        # cursor = conn.execute('SELECT * FROM boards WHERE description = ? AND version = ?', (board.description,board.version))
        # cursor = conn.execute('SELECT * FROM boards WHERE description = ?', (board.description,))
        qry = f"SELECT * FROM boards WHERE description in ( '{board.description}' , 'Generic ' || '{board.description}')"
        cursor = conn.execute(qry)
        row = cursor.fetchone()
        if row:
            row = dict(row)
            if row["version"] != board.version:
                # print(f"version mismatch: {row['version']} != {board.version}")
                pass
        else:
            json = Path("../micropython") / board.path.parent / "board.json"
            if json.exists():
                missing.append(board.board_id)
                print(f"{board.board_id} {board.version} {board.description} from {board.path} not found")
            else:
                # print(f"{board.board_id} -no json")
                pass

print(f"Missing {missing} boards")

SAMD_GENERIC_D21X18 v1.25.0-preview Generic SAMD21J18 with SAMD21J18A from ports\samd\boards\SAMD_GENERIC_D21X18\mpconfigboard.h not found
SAMD_GENERIC_D51X19 v1.25.0-preview Generic SAMD51P19 with SAMD51P19A from ports\samd\boards\SAMD_GENERIC_D51X19\mpconfigboard.h not found
SAMD_GENERIC_D51X20 v1.25.0-preview Generic SAMD51P20 with SAMD51P20A from ports\samd\boards\SAMD_GENERIC_D51X20\mpconfigboard.h not found
ADAFRUIT_NEOKEY_TRINKEY v1.25.0-preview NeoKey Trinkey with SAMD21E18A from ports\samd\boards\ADAFRUIT_NEOKEY_TRINKEY\mpconfigboard.h not found
ADAFRUIT_QTPY_SAMD21 v1.25.0-preview QT Py with SAMD21E18A from ports\samd\boards\ADAFRUIT_QTPY_SAMD21\mpconfigboard.h not found
RPI_PICO2_W v1.25.0-preview Raspberry Pi Pico 2 W from ports\rp2\boards\RPI_PICO2_W\mpconfigboard.h not found
SPARKFUN_IOTNODE_LORAWAN_RP2350 v1.25.0-preview SparkFun IoT Node LoRaWAN from ports\rp2\boards\SPARKFUN_IOTNODE_LORAWAN_RP2350\mpconfigboard.h not found
SPARKFUN_PROMICRO_RP2350 v1.25.0-preview Spark

In [1]:
from mpflash.vendor.board_database import Database
from pathlib import Path

db = Database(Path("../micropython"))

list_of_boards = db.boards

In [10]:
for b in missing:
    print(db.boards[b])

Board(name='SAMD_GENERIC_D21X18', variants=[], url='', mcu='samd21', product='Generic SAMD21J18', vendor='Microchip', images=['generic_board.jpg'], deploy=['../deploy.md'], port=Port(name='samd'), path='../micropython/ports/samd/boards/SAMD_GENERIC_D21X18')
Board(name='SAMD_GENERIC_D51X19', variants=[], url='', mcu='samd51', product='Generic SAMD51P19', vendor='Microchip', images=['generic_board.jpg'], deploy=['../deploy.md'], port=Port(name='samd'), path='../micropython/ports/samd/boards/SAMD_GENERIC_D51X19')
Board(name='SAMD_GENERIC_D51X20', variants=[], url='', mcu='samd51', product='Generic SAMD51P20', vendor='Microchip', images=['generic_board.jpg'], deploy=['../deploy.md'], port=Port(name='samd'), path='../micropython/ports/samd/boards/SAMD_GENERIC_D51X20')
Board(name='ADAFRUIT_NEOKEY_TRINKEY', variants=[], url='https://www.adafruit.com/product/5020', mcu='samd21', product='NeoKey Trinkey', vendor='Adafruit', images=['neokey_trinkey.jpg'], deploy=['../deploy.md'], port=Port(name=

In [ ]:
db = Database(mpy_dir)
shortlist = list(iter_boards(db, version=version))
print(f"boards found {len(db.boards.keys())}")